### Presentation Types:  Fuga, ID, and PEN

* **Updated June 2022**
* **Key Features**:

    * Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
    * For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### Load the Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [2]:
prefix = 'https://crimproject.org/mei/'
# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0001.mei'
url = prefix + mei_file

piece = importScore(url)


print(piece.metadata)


Successfully imported https://crimproject.org/mei/CRIM_Model_0001.mei
{'title': 'Veni speciosam', 'composer': 'Johannes Lupi'}


## Find Presentation Types
* `piece.presentationTypes()`

- limit to entries (following rests or section) = `limit_to_entries = True`.
- allowing 'moving window' of all patterns in every voice = `limit_to_entries = False`
- set the length of the soggetti with `melodic_ngram_length = n`
- set the maximum difference between similar soggetti with `edit_distance_threshold = n`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types = True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types = False`

* For example:

`piece.presentationTypes(limit_to_entries = True,
                        edit_distance_threshold = 0,
                        include_hidden_types = False,
                        combine_unisons = True,
                       melodic_ngram_length = 4)`
                       
                       
* Read the documentation:  `print(piece.presentationTypes.__doc__)`

In [4]:

piece.presentationTypes(limit_to_entries = True,
                        edit_distance_threshold = 1,
                        include_hidden_types = True,
                        combine_unisons = False,
                       melodic_ngram_length = 4)


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Johannes Lupi,Veni speciosam,0.0,"[1/1.0, 2/3.0, 5/1.0, 6/3.0]","[P-4, P-5, P-4]","[0.0, 12.0, 32.0, 44.0]","[(5, -2, 2, 3), (4, -2, 2, 3)]","[12.0, 20.0, 12.0]","[Superius, Contratenor, PrimusTenor, Bassus]",ID,4,True
1,Johannes Lupi,Veni speciosam,0.0,"[1/1.0, 2/3.0, 5/1.0, 6/3.0, 9/2.0, 10/1.0]","[P-4, P-5, P-4, P4, P8]","[0.0, 12.0, 32.0, 44.0, 66.0, 72.0]","[(5, -2, 2, 3), (4, -2, 2, 3)]","[12.0, 20.0, 12.0, 22.0, 6.0]","[Superius, Contratenor, PrimusTenor, Bassus, S...",FUGA,6,True
2,Johannes Lupi,Veni speciosam,94.0,"[12/4.0, 16/4.0]",[P4],"[94.0, 126.0]","[(-3, 2, 2, -2)]",[32.0],"[Bassus, Bassus]",FUGA,2,False
3,Johannes Lupi,Veni speciosam,138.0,"[18/2.0, 19/4.0, 24/2.0, 25/4.0]","[P1, P1, P1]","[138.0, 150.0, 186.0, 198.0]","[(-2, -3, 2, 2)]","[12.0, 36.0, 12.0]","[PrimusTenor, SecundusTenor, PrimusTenor, Secu...",ID,4,False
4,Johannes Lupi,Veni speciosam,138.0,"[18/2.0, 18/4.0, 20/4.0, 21/2.0]","[P-4, P8, P5]","[138.0, 142.0, 158.0, 162.0]","[(-2, -3, 2, 2)]","[4.0, 16.0, 4.0]","[PrimusTenor, Bassus, Contratenor, Superius]",ID,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...
115,Johannes Lupi,Veni speciosam,978.0,"[122/2.0, 123/2.0, 128/4.0, 129/4.0]","[P8, M-9, P1]","[978.0, 986.0, 1030.0, 1038.0]","[(2, -2, -3, 2)]","[8.0, 44.0, 8.0]","[SecundusTenor, Superius, Contratenor, Secundu...",ID,4,False
116,Johannes Lupi,Veni speciosam,978.0,"[122/2.0, 122/4.0, 123/2.0, 125/1.0, 126/4.0, ...","[P1, P8, P-5, P-5, P-8, P8, P1]","[978.0, 982.0, 986.0, 1000.0, 1014.0, 1020.0, ...","[(2, -2, -3, 2)]","[4.0, 4.0, 14.0, 14.0, 6.0, 10.0, 8.0]","[SecundusTenor, PrimusTenor, Superius, Contrat...",FUGA,8,False
117,Johannes Lupi,Veni speciosam,978.0,"[122/2.0, 122/4.0, 123/2.0]","[P1, P8]","[978.0, 982.0, 986.0]","[(2, -2, -3, 2)]","[4.0, 4.0]","[SecundusTenor, PrimusTenor, Superius]",PEN,3,False
118,Johannes Lupi,Veni speciosam,982.0,"[122/4.0, 125/1.0, 127/3.0, 129/4.0]","[P4, P-12, P8]","[982.0, 1000.0, 1020.0, 1038.0]","[(2, -2, -3, 2)]","[18.0, 20.0, 18.0]","[PrimusTenor, Contratenor, Bassus, SecundusTenor]",ID,4,False


## Find Presentation Types in Several Pieces at Once

* Results are combined into a single dataframe
* Set thresholds and other features as noted above

* Read the documentation:  `print(CorpusBase.batch.__doc__)`

In [16]:

corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0006_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0006_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0006_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0006_4.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0006_5.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0007_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0007_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0007_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0007_4.mei', 
                     'https://crimproject.org/mei/CRIM_Mass_0007_5.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Successfully imported https://crimproject.org/mei/CRIM_Mass_0007_5.mei
Memoized piece detected.


In [17]:
func = ImportedPiece.presentationTypes
list_of_dfs = corpus.batch(func=func, kwargs={'edit_distance_threshold' : 1,
                        'include_hidden_types' : True,
                        'combine_unisons' : True,
                       'melodic_ngram_length' : 4}, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=False)


In [18]:
combined_df

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Jean Guyon,Missa Je suis déshéritée: Kyrie,34.0,"[5/2.0, 6/3.0, 10/4.0, 12/3.0, 15/1.0]","[m13, P-15, P15, m-13]","[34.0, 44.0, 78.0, 92.0, 124.0]","[(-2, -2, -3, 2), (-2, -2, -2, 2)]","[10.0, 34.0, 14.0, 32.0]","[[Bassus], [Superius], [Bassus], [Superius], [...",FUGA,5,True
1,Jean Guyon,Missa Je suis déshéritée: Kyrie,124.0,"[15/1.0, 17/3.0, 19/2.0]","[P8, P-8]","[124.0, 144.0, 158.0]","[(-2, -2, -2, -2), (-2, -2, -2, -3)]","[20.0, 14.0]","[[Tenor], [Superius], [Bassus]]",FUGA,3,True
2,Jean Guyon,Missa Je suis déshéritée: Kyrie,134.0,"[16/2.0, 17/4.0, 21/2.0, 24/1.0]","[P1, P8, P-11]","[134.0, 146.0, 174.0, 196.0]","[(2, 3, -2, 2)]","[12.0, 28.0, 22.0]","[[Contratenor], [Tenor], [Superius], [Bassus]]",FUGA,4,False
3,Jean Guyon,Missa Je suis déshéritée: Kyrie,182.0,"[22/2.0, 26/1.0]",[P8],"[182.0, 212.0]","[(2, 3, -2, -2)]",[30.0],"[[Tenor], [Superius]]",FUGA,2,False
4,Jean Guyon,Missa Je suis déshéritée: Kyrie,254.0,"[31/2.0, 31/4.0, 32/2.0, 34/1.0, 34/4.0]","[P8, P-12, P5, P1]","[254.0, 258.0, 262.0, 276.0, 282.0]","[(3, -2, -2, -2), (4, -2, -2, -2)]","[4.0, 4.0, 14.0, 6.0]","[[Tenor], [Superius], [Bassus], [Contratenor],...",FUGA,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...
24,Nicolas Gombert,Missa Je suis déshéritée: Agnus Dei,458.0,"[57/2.0, 61/4.0]",[P1],"[458.0, 494.0]","[(-3, 2, 2, 2)]",[36.0],"[[Superius 1], [Superius 1]]",FUGA,2,False
0,Pierre Cadéac,Je suis déshéritée,26.0,"[4/2.0, 12/2.0]",[P1],"[26.0, 90.0]","[(2, -3, -2, -3)]",[64.0],"[Bassus, Bassus]",FUGA,2,False
1,Pierre Cadéac,Je suis déshéritée,34.0,"[5/2.0, 7/2.0, 13/2.0, 15/2.0]","[P8, P-8, P8]","[34.0, 50.0, 98.0, 114.0]","[(-2, -2, -2, 2)]","[16.0, 48.0, 16.0]","[Tenor, Superius, Tenor, Superius]",ID,4,False
2,Pierre Cadéac,Je suis déshéritée,134.0,"[17/4.0, 19/2.0]",[P8],"[134.0, 146.0]","[(2, 3, -2, 2)]",[12.0],"[Tenor, Superius]",FUGA,2,False


#### Save to your folder of CSV's here in the Jupyter Hub
#### You can save as CSV, or as Excel.
#### You will then need to download this to your computer to view it properly

* Note that in the following part of the code below, you will need to give your file a name:

`saved_csv/**file_name**.xlsx`

In [26]:
writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')
points.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [6]:
points.to_csv('saved_csv/your_file_title.csv')